In [26]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 
from collections import Counter
morph = MorphAnalyzer()

c:\program files\python35\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\program files\python35\lib\site-packages\_pytest\fixtures.py:834: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
c:\program files\python35\lib\site-packages\_pytest\fixtures.py:836: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

#### Данные

### 1) Нормализация

In [27]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [28]:
texts = open('habr_texts.txt', "r", encoding="utf-8").read().splitlines()

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [29]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

### 1.1) Дополнительная нормализация 

In [30]:
def preprocess(text_words):
    return [i for i in text_words if str.isalnum(i)]

In [31]:
texts = [preprocess(text) for text in texts]

### 2) Добавим n-grams

In [32]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.5) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

c:\program files\python35\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
c:\program files\python35\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
c:\program files\python35\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
c:\program files\python35\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
c:\program files\pyt

In [208]:
for i in ngrammed_texts:
    print(i[:100])

#### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [44]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [45]:
from nltk.corpus import stopwords
from six import iteritems

### 3.1) Делаем хороший словарь 

убираем русские и английские стоп-слова, подбираем порог (не помню, какой был, но мне понравился no_above=0.3, no_below=30)

In [47]:
en_words = stopwords.words('english')
ru_words = stopwords.words('russian')
stoplist = ["gt", "lt"] + en_words + ru_words
#dictinary.filter_extremes(no_above=0.3, no_below=30)
stop_ids = [dictinary.token2id[stopword] for stopword in stoplist if stopword in dictinary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictinary.dfs) if docfreq == 1]
dictinary.filter_tokens(stop_ids + once_ids) 
dictinary.compactify()

In [49]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [53]:
print(dictinary)

Dictionary(6379 unique tokens: ['нередко', 'достигать', 'непросто', 'быстро', 'ключевой_слово']...)


Преобразуем наши тексты в мешки слов. 

In [51]:
corpus = [dictinary.doc2bow(text) for text in texts]
# если текстов много, то тут может быть генератор

### 4) Подберем правильные параметры

In [58]:
import warnings
warnings.filterwarnings(action='once')

In [64]:
eta = [0.1, 0.3, 0.5, 0.7, 0.9]
alpha = [0.1, 0.3, 0.5, 0.7, 0.9]
passes = [5, 10, 15]
topics = [20, 50, 100, 200]

for e in eta:
    for a in alpha:
        for p in passes:
            for t in topics:
                lda = gensim.models.LdaMulticore(corpus, num_topics=t, id2word=dictinary, passes=p, eta=e, alpha=a)
                print("*"*10)
                print(e,a,p,t)
                topics_ = lda.print_topics()
                for ti in topics_:
                    tops = re.findall('"(.+?)"', ti[1])
                    print("{}) ".format(ti[0]+1) + " ".join(tops))
                print("="*10)
                print(lda.log_perplexity(corpus))
                print("="*10)
                perplex = lda.bound(corpus)
                print(perplex)
                print("="*10)
                coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                texts=texts, 
                                                dictionary=dictinary, coherence='c_v')
                topics__ = []
                for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
                    topic = [word for word, _ in topic]
                    topics__.append(topic)
                coherence_model_lda = gensim.models.CoherenceModel(topics=topics__, 
                                                texts=texts, 
                                                dictionary=dictinary, coherence='c_v')
                print(coherence_model_lda.get_coherence())
                print("*"*10)

c:\program files\python35\lib\site-packages\gensim\models\ldamodel.py:853: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


**********
0.1 0.1 5 20
1) база кластер сервер запрос язык память вакансия программирование ошибка программа
2) файл windows память устройство программа ядро диск linux запись драйвер
3) сервис продукт клиент технология безопасность управление услуга инфраструктура доступ облачный
4) сервер файл сеть адрес пакет настройка виртуальный база клиент машина
5) игра игрок игровой вселенная объект играть её vr энергия галактика
6) end begin объект result string инструмент var анализатор function ошибка
7) сайт google angular домен реклама аккаунт бэкап ссылка резервный объект
8) значение алгоритм модель массив точка набор блок сеть вектор элемент
9) технология космический рынок аппарат доклад земля развитие спутник участник марс
10) сайт клиент деньга цена скидка карта сервер месяц рубль продажа
11) android элемент страница список интерфейс экран сайт блок браузер кнопка
12) мозг звук сигнал исследование учёный пациент область клетка устройство наушник
13) ms объект элемент класс метод точка 

-7.76233072453694
-15818631.086899102
0.5796001098837058
**********
**********
0.1 0.1 10 50
30) мозг пациент глаз исследование движение рука область боль операция болезнь
9) позиция apple буква gpu компьютер память собственный лишь камень элемент
11) услуга анализ список сайт элемент клиент поддержка модель коллекция форум
37) язык библиотека инструмент строка javascript страница элемент программирование тест браузер
17) значение переменный сумма рубль канал документ доход поль кнопка налог
46) файл настройка скрипт user добавить сервер папка запустить установка модуль
27) технология рынок сервис клиент мобильный робот сеть платформа услуга интернет
19) клиент письмо сеть сотрудник crm рассылка продажа адрес план почта
33) std amp const указатель template int return контейнер auto поток
34) тестирование тест продукт ошибка требование этап интерфейс сайт программа проектирование
32) сервер виртуальный сеть инфраструктура машина оборудование облачный управление облако диск
43) блок анал

-7.70727656668777
-15706406.630759716
0.5799497184987673
**********
**********
0.1 0.1 15 100
46) боль уязвимость пациент обновление менеджер мозг атаковать нерв привилегия ощущение
36) устройство телефон модель плата питание кабель режим корпус модуль компьютер
23) технология безопасность сша организация блокчейн российский защита миллион страна крупный
87) user id файл запрос post amp http name добавить url
51) вкладка робот screen терминал симулятор запустить панель добавить рассмотреть нажать
28) сайт google уязвимость блокировка сеть атака аккаунт android ресурс поиск
91) сервер сайт база сервис запрос нагрузка php visual отчёт studio
71) скидка товар цена заказ сервер акция энергия стоимость хостинг месяц
17) производство печать принтер деталь лампа корпус 3d машина устройство транзистор
11) файл библиотека программа модуль ошибка строка страница плагин компилятор сборка
78) свет звезда прибор солнце цвет цвета красный источник яркость яркий
66) класс объект метод object class зн

-7.999320633179462
-16301555.19614172
0.5370233845890512
**********
**********
0.1 0.3 5 200
156) миллион рейтинг видеонаблюдение выручка производитель игра голос ruby итог 2016
87) технология безопасность устройство атака защита рынок сеть доступ деньга интернет
86) драйвер сетевой лампа очередь пакет прерывание cpu устройство обработка протокол
168) вода банка финансовый оборудование океан охлаждение цод кровь банк животное
178) элемент запись iphone название признак значение поиск select наименование список
130) сертификат сервер хост ключ файл защита план политика шифрование центр
67) массив case репликация режим title технология продукт ядро асинхронный набор
108) метод класс объект return запрос файл библиотека состояние значение var
45) сервер сайт сервис язык технология function php страница связь база
72) нативный int unsigned класс struct обёртка библиотека управляемый пакет указатель
199) фильм астероид type рейтинг фигура земля bytes орбита зона корабль
143) пароль intel хр

...

-7.7485793244465695
-15790581.28211081
0.59197488690043
**********
**********
0.3 0.1 10 50
12) windows microsoft server виртуальный вм бот домен сервер хост azure
19) type std список поддержка таблица добавить база конфигурация коммит storage
37) память процессор производительность тест ядро intel нагрузка поток запрос архитектура
30) устройство смартфон google apple рынок телефон android мобильный amazon ноутбук
44) страна деньга российский сайт миллион сша закон сеть россия блокчейн
1) string import new def url public http void java private
8) ошибка анализатор регистр amp значение строка int переменный библиотека проверка
34) сеть трафик маршрутизатор маршрут пакет метка интерфейс ip данный адрес
17) боль лекция uber рейтинг сервис токен водитель пассажир основа платформа
14) end begin property else integer nil boolean модуль язык var
25) язык текст элемент документ перевод страница option интерфейс правило документация
39) модель телефон клавиатура экран доставка точка подсветка д

-7.686839322329158
-15664734.505341291
0.5883507711892616
**********
**********
0.3 0.1 15 100
5) технология блокчейн автомобиль платформа сеть финансовый адрес открытый цифровой ibm
25) сигнал частота измерение искажение импульс выход значение источник спектр обратный
78) мозг исследование учёный пациент клетка жизнь боль организм болезнь тело
11) триггер архив монета windows параметр комбинация ошибка папка скрипт name
72) usb часы механизм устройство кабель режим фоновый порт тренировка пк
2) свет сцена текстура карта тень рендеринг тенить объект пиксель дорога
35) звук наушник музыка звуковой усилитель акустический микрофон частота устройство громкость
90) ветка коммит master стенд flow git фон объектив база метод
65) swift метод трансляция робот протокол класс ios значение доступ let
81) сайт российский проектирование продукт страница прототип программный требование 1с этап
7) angular лицо когнитивный убеждение мнение искажение факт распознавание эффект очки
14) function return ко

-7.943322639546636
-16187440.137766719
0.5214323825097614
**********
**********
0.3 0.3 5 200
196) камера полёт скорость аппарат прибор телескоп ветер видео съёмка высота
122) ms sum mb php производительность тест javascript язык реальный сравнение
86) end begin var function else sudo property git integer string
68) страница сообщение текст элемент строка сайт ссылка https карта кнопка
52) мужчина женщина подарок мозг различие стартовый стол длина игра регион
110) номер звонок оператор вызов сервер связь абонент null телефон клиент
133) аккаунт коммерческий декабрь 2016 новость дистрибутив название сегодня google цель
169) директива void ядро токен amp обработка объект запрос метод const
180) конфигурация configuration ресурс end пост кеш файл 2017 php потомок
25) напряжение ток питание плата станок мощность схема астероид аккумулятор устройство
155) язык let input return ios поддержка android ruby инструмент php
99) public метод объект класс new return string var user private
34) кноп

-7.967076457619537
-16235854.795422435
0.5161151127332321
**********
**********
0.3 0.3 15 20
1) клиент продукт сайт продажа месяц деньга понять заказчик сервис услуга
2) ms материал напряжение устройство датчик мощность модель ток research производство
3) return amp function int end var else string const true
4) таблица select запрос робот null элемент база строка индекс значение
5) public new string void private return class null android int
6) сеть безопасность доступ трафик интернет атака защита устройство связь сервис
7) вселенная энергия звезда галактика файл объект квантовый теория свет частица
8) сайт сервер сообщение страница устройство браузер домен файл кнопка сертификат
9) сервер виртуальный диск производительность база машина память ядро поддержка резервный
10) аппарат космический земля спутник скидка энергия температура 2016 цена полёт
11) сотрудник бизнес деньга исследование технология рынок специалист крупный пациент развитие
12) устройство звук модель камера смартфон э

KeyboardInterrupt: 

Посмотрим на топики.

### 5) Я попробовала несколько моделей, лучшими оказались LdaMulticore с параметрами:

#### eta = [0.1, 0.3]
#### alpha = [0.1, 0.3, 0.5]
#### passes = 15
#### topics = [20, 50]

Выше, собственно, все результаты со всякими параметрами. Приведу самые классные темы из LdaMulticore( eta=0.1, alpha=0.5, passes=15, topics=50)

1) земля космический спутник аппарат орбита планета километр марс солнечный астероид

2) модель обучение research нейросеть изображение машинный science ветка сеть набор

3) запрос select user таблица end база sql post create id

4) цена скидка товар рубль магазин ноутбук телефон модель смартфон экран

5) устройство windows управление драйвер linux сетевой настройка пакет ядро доступ


In [224]:
topics = lda.print_topics()

Это я пробовала обучать модель с парамтерами LdaMulticore(passes=5, topics=20) получилось очень годно

In [225]:
for t in topics:
    tops = re.findall('"(.+?)"', t[1])
    print("{}) ".format(t[0]+1) + " ".join(tops))

1) значение элемент int алгоритм точка блок массив память размер метод
2) продукт сотрудник клиент язык опыт программист друг программирование заниматься доклад
3) устройство сервер сеть клиент технология услуга сервис управление облачный инфраструктура
4) user сервер машина виртуальный server хост вм windows кластер база
5) сеть исследование обучение модель технология учёный метод мозг область изображение
6) react компонент язык уязвимость javascript div css angular боль элемент
7) файл программа объект папка галактика строка размер параметр устройство диск
8) файл сообщение ключ сервер android настройка пароль добавить клиент сервис
9) товар скидка страна сервис россия деньга сайт сша продажа миллион
10) космический значение земля станция аппарат спутник орбита переменный канал программа
11) устройство камера смартфон экран модель режим сигнал видео звук телефон
12) research резервный go копия сервер science amp объём хостинг память
13) сайт карта страница список глаз элемент анимаци

In [172]:
for t in topics:
    tops = re.findall('"(.+?)"', t[1])
    print("{}) ".format(t[0]+1) + " ".join(tops))

1) ошибка файл android react research библиотека type язык message amp
2) вселенная страница земля энергия звезда элемент печать объект галактика планета
3) ген клетка машина болезнь метод компьютер математика float программа язык
4) сервер клиент резервный управление питание нагрузка напряжение база оборудование режим
5) товар деньга скидка месяц рынок продажа цена рубль магазин покупка
6) значение кнопка переменный игрок состояние поль точка канал игра пол
7) программа игра компьютер диск участник производство игрок цена город курс
8) файл amp настройка ключ сервер пароль пакет адрес список windows
9) public return класс string new метод class function amp объект
10) сообщение файл сервер игра пакет интерфейс сеть библиотека маршрутизатор запрос
11) устройство безопасность запись доступ файл атака сеть уязвимость программа база
12) сигнал анимация элемент движение метод значение параметр частота усилитель объект
13) сайт точка страница обучение запрос сеть алгоритм база мозг вектор
1

### Выводы: количество топиков должно быть 20-50, passes не меньше 15, с eta и alpha можно поиграться (сделаю, если останется время). Для добавления tfIdf буду использовать параметры  LdaMulticore(eta=0.1, alpha=0.5, passes=15, topics=50)

In [145]:
pyLDAvis.enable_notebook()

In [146]:
pyLDAvis.gensim.prepare(lda, corpus, dictinary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
19     9.897190        1       1  0.099480 -0.019283
16     7.772626        1       2  0.052553 -0.044934
13     7.554558        1       3 -0.121144  0.060404
18     7.156407        1       4  0.074393  0.026842
8      6.891493        1       5 -0.182887  0.002991
6      6.846927        1       6  0.097405  0.002912
7      6.660067        1       7 -0.081100 -0.043937
12     5.982779        1       8  0.017163  0.023907
14     5.613823        1       9  0.053925 -0.027467
3      4.766638        1      10  0.005890 -0.033942
4      4.616460        1      11  0.079584 -0.045859
10     4.539985        1      12 -0.003185 -0.067861
9      4.354466        1      13 -0.049177 -0.018401
1      3.808119        1      14  0.053889  0.062226
5      3.691137        1      15  0.010875  0.104234
0      3.091127        1      16 -0.081104 -0.021795
11     2.578069        1      17  0.024058  0.059424
2      1.421987        1      18  0.013184  0.023481
15     1.389966        1      19 -0.006568 -0.006936
17     1.366178        1      20 -0.057232 -0.036006, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
1623  Default  4119.000000         return  4119.000000  30.0000  30.0000
6203  Default  5154.000000           игра  5154.000000  29.0000  29.0000
1231  Default  1321.000000            end  1321.000000  28.0000  28.0000
1681  Default  2432.000000         public  2432.000000  27.0000  27.0000
1433  Default  5910.000000           файл  5910.000000  26.0000  26.0000
5270  Default  3215.000000            int  3215.000000  25.0000  25.0000
854   Default  4238.000000            amp  4238.000000  24.0000  24.0000
6303  Default  2559.000000         string  2559.000000  23.0000  23.0000
1592  Default  4814.000000         сервер  4814.000000  22.0000  22.0000
4235  Default  4838.000000       значение  4838.000000  21.0000  21.0000
6349  Default  1169.000000            set  1169.000000  20.0000  20.0000
2997  Default  5429.000000     устройство  5429.000000  19.0000  19.0000
1013  Default  2303.000000            new  2303.000000  18.0000  18.0000
1561  Default  1714.000000           null  1714.000000  17.0000  17.0000
323   Default  4267.000000          метод  4267.000000  16.0000  16.0000
913   Default  1049.000000           user  1049.000000  15.0000  15.0000
300   Default  3216.000000          класс  3216.000000  14.0000  14.0000
6217  Default  2062.000000       function  2062.000000  13.0000  13.0000
452   Default  1536.000000          class  1536.000000  12.0000  12.0000
1774  Default  1717.000000         сигнал  1717.000000  11.0000  11.0000
631   Default  1421.000000           else  1421.000000  10.0000  10.0000
4854  Default  3961.000000         объект  3961.000000   9.0000   9.0000
2563  Default   719.000000         доклад   719.000000   8.0000   8.0000
2854  Default  1629.000000           void  1629.000000   7.0000   7.0000
3324  Default  3401.000000        элемент  3401.000000   6.0000   6.0000
3182  Default  2673.000000         строка  2673.000000   5.0000   5.0000
517   Default  1736.000000            var  1736.000000   4.0000   4.0000
6308  Default   514.000000          begin   514.000000   3.0000   3.0000
2100  Default  1905.000000     переменный  1905.000000   2.0000   2.0000
3942  Default  1710.000000          игрок  1710.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
5569  Topic20    71.843835       username   260.122057   3.0065  -5.9598
42    Topic20   122.230782         create   466.837072   2.9531  -5.4284
5512  Topic20   166.740622           test   646.854296   2.9375  -5.1178
3170  Topic20    77.483441            max   312.908894   2.8973  -5.8842
5054  Topic20    60.814072            nil   265.115482   2.8208  -6.1264
391   Topic20    

### 6) Добавим TfIdf

In [65]:
tfidfmodel = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus_tfidf = tfidfmodel[corpus]

### 7) Обучим модельку

In [94]:
lda2 = gensim.models.LdaMulticore(corpus_tfidf, 50, id2word=dictinary, passes=15, eta=0.1, alpha=0.5) # если поддерживается многопоточность
topics = lda2.print_topics()
for t in topics:
    tops = re.findall('"(.+?)"', t[1])
    print("{}) ".format(t[0]+1) + " ".join(tops))

c:\program files\python35\lib\site-packages\gensim\models\ldamodel.py:853: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


50) криптовалюта вендор велосипед client мс окно лазерный пятница модификация option
11) echo подпись платёжный урок postgresql config admin монтаж потенциально позиция
23) владелец override слышать искажение беспилотный юридический орган луна налог рейтинг
2) доставка регистр date unsigned sdk странный flash главное получатель mobile
22) std void врач скидка static экземпляр studio сессия выяснить купить
21) ibm повышение вес сущность реакция школа status новичок подписчик нарушение
44) amazon программирование facebook ребёнок руководство ос плюс читать будущее рассказывать
48) причина тело программист интересно квантовый дерево понимание задержка предложение обратный
14) измерение глава век заблокировать расчёт спрайт info интерактивный полностью доступно
10) iphone сегмент сократить яркость индикатор счётчик 400 вводить dom выдать
3) космический станция температура звезда вселенная научный поверхность галактика фильм двигатель
42) data print text file cisco оба отображать request fu

In [95]:
lda2.log_perplexity(corpus_tfidf)

c:\program files\python35\lib\site-packages\gensim\models\ldamodel.py:853: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


-11.192284789234824

In [96]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda2, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')
topics_tf = []
for topic_id, topic in lda2.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics_tf.append(topic)
coherence_model_lda2 = gensim.models.CoherenceModel(topics=topics_tf, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')
coherence_model_lda2.get_coherence()

0.31706957289827786

### 8) Что изменилось?

#### С tfidf стало как-то хуже, например, появились темы, содержащие достаточно много тех же ключевых слов:

#### значения перплексии и когерентности для обеих моделей:

без tf idf :

log = -7.79

coherence = 0.6



с tf idf :

log = -17.90
coherence = 0.56

### Разложение матриц в sklearn

In [71]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import nltk

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [83]:
stexts = [' '.join([t for t in text if t not in stoplist]) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [84]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

c:\program files\python35\lib\site-packages\numpy\matrixlib\defmatrix.py:68: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


Разложим её.

In [91]:
n_nmf = 50

In [85]:
model = NMF(n_components=n_nmf)

In [86]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [92]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(n_nmf):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [93]:
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 41,Topic # 42,Topic # 43,Topic # 44,Topic # 45,Topic # 46,Topic # 47,Topic # 48,Topic # 49,Topic # 50
0,сотрудник,объект,игра,обучение,сигнал,public,камера,космический,атака,react,...,amp,тест,лекция,автомобиль,устройство,php,текстура,книга,облачный,игрок
1,продукт,класс,игровой,нейросеть,частота,void,видео,спутник,уязвимость,компонент,...,int,тестирование,курс,беспилотный,смартфон,symfony,карта,читать,облако,игра
2,программист,метод,играть,нейронный,напряжение,string,регистратор,орбита,безопасность,props,...,std,тестировать,студент,tesla,телефон,laravel,изображение,чтение,инфраструктура,игровой
3,деньга,значение,steam,машинный,ток,private,blackvue,марс,злоумышленник,div,...,void,тестовый,университет,водитель,мобильный,yii,шейдёр,автор,сервис,враг
4,бизнес,свойство,unity,алгоритм,плата,new,съёмка,аппарат,вредоносный,jsx,...,char,ошибка,программирование,машина,apple,прислать,точка,прочитать,услуга,комната
5,опыт,person,движок,сеть,питание,static,смартфон,земля,защита,dom,...,struct,тестировщик,видео,автопилот,аккумулятор,yii2,свет,идея,технология,персонаж
6,зарплата,вызов,nintendo,интеллект,датчик,class,движение,луна,пароль,элемент,...,return,test,вступительный,дорога,умный,api,координата,дизайн,оборудование,миссия
7,менеджер,object,жанр,изображение,мощность,return,ivideon,наса,ботнет,render,...,else,библиотека,школа,uber,гаджет,массив,пиксель,дизайнер,цод,лига
8,руководитель,переменный,приставка,модель,усилитель,null,кадр,станция,угроза,компонента,...,unsigned,модульный,занятие,крыша,девайс,фреймворка,анимация,навык,iaas,играть
9,идея,реализация,мобильный,искусственный,схема,override,видеонаблюдение,полёт,доступ,input,...,указатель,баг,профессор,ibm,беспроводный,релиз,освещение,глава,платформа,форум


### 10) Мне кажется, у NMF темы тоже классные, есть, например, тема 6 (очень похожа на джаву), есть тема 42 про тестирование